In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
205,AFG,Asia,Afghanistan,2020-08-02,36710.0,0.0,1283.0,0.0,943.015,0.000,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
352,ALB,Europe,Albania,2020-08-02,5396.0,120.0,161.0,4.0,1875.043,41.699,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
563,DZA,Africa,Algeria,2020-08-02,30950.0,556.0,1223.0,13.0,705.798,12.679,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-01,France,189148.0,0.0,30303.0,0.0,0.0,0.0
0,2020-08-01,China,87826.0,0.0,4669.0,0.0,0.0,0.0
0,2020-08-01,Italy,248003.0,0.0,35167.0,0.0,0.0,0.0
0,2020-08-01,Spain,290961.0,0.0,28456.0,0.0,0.0,0.0
0,2020-08-01,United States,4637866.0,0.0,154624.0,0.0,0.0,0.0
0,2020-08-01,World,17888151.0,0.0,686537.0,0.0,0.0,0.0
0,2020-08-01,United Kingdom,304219.0,0.0,46206.0,0.0,0.0,0.0
0,2020-08-01,Germany,210495.0,0.0,9161.0,0.0,0.0,0.0
0,2020-08-01,Iran,307728.0,0.0,16982.0,0.0,0.0,0.0
0,2020-08-01,Turkey,231781.0,0.0,5705.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-02,France,189148.0,187919.0,30303.0,30265.0,1229.0,38.0
0,2020-08-02,China,87826.0,87820.0,4669.0,4667.0,6.0,2.0
0,2020-08-02,Italy,248003.0,247832.0,35167.0,35146.0,171.0,21.0
0,2020-08-01,Spain,290961.0,288522.0,28456.0,28445.0,2439.0,11.0
0,2020-08-02,United States,4637866.0,4620444.0,154624.0,154447.0,17422.0,177.0
0,2020-08-02,World,17888151.0,17841669.0,686537.0,685281.0,46482.0,1256.0
0,2020-08-02,United Kingdom,304219.0,303952.0,46206.0,46193.0,267.0,13.0
0,2020-08-02,Germany,210495.0,209893.0,9161.0,9141.0,602.0,20.0
0,2020-08-02,Iran,307728.0,306752.0,16982.0,16982.0,976.0,0.0
0,2020-08-02,Turkey,231781.0,231869.0,5705.0,5710.0,-88.0,-5.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")